<h1 dir=rtl align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
    SharifML Contest - ML
</font>
</h1>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
   
</font>
</p>

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معرفی مجموعه داده
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    در فایل اولیه‌ی این سوال دو مجموعه‌داده با نام‌های <code>train_dataset</code> و <code>test_dataset</code> قرار دارد که به ترتیب مجموعه‌داده مربوط به آموزش و آزمون هستند. این دو به‌ترتیب شامل ۲۵۹۴۱ و ۱۷۰۰ سطر هستند.
    <br>
    ستون‌های این فایل به شرح زیر است:
</font>
</p>

<center>
<div dir=rtl align=center style="direction: rtl;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>

| <b>نام ویژگی</b> | <b>توضیح ویژگی</b> |
| :---: | :---: |
| <code>Class</code> | ستون هدف که کلاس مربوط به آن گونه است.|
| <code>Others</code> | ویژگی‌هایی که می‌توانید از آنها برای آموزش مدل خود استفاده کنید. |

</font>

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
خواندن مجموعه‌داده
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>

</font>
</p>

In [1]:
import numpy as np
import pandas as pd
import ast
import re

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier

In [2]:
train_df = pd.read_csv('train_dataset.csv')
test_df = pd.read_csv('test_dataset.csv')

In [3]:
def extract_numeric(value):
    if pd.isna(value):
        return np.nan

    if not isinstance(value, str):
        return value
    
    if value.strip == "":
        return np.nan

    match = re.match(r"([0-9.]+)", value.strip())
    if match:
        return float(match.group(1))
    
    return value

def preprocess_data(train, test, dict_columns=[]):
    for col in dict_columns:
        if col in train.columns:
            train[col] = train[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
            dict_train = pd.json_normalize(train[col])
            dict_train.columns = [f"{col}_{key}" for key in dict_train.columns]
            train = pd.concat([train, dict_train], axis=1)
            train.drop(columns=[col], inplace=True)

        if col in test.columns:
            test[col] = test[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
            dict_test = pd.json_normalize(test[col])
            dict_test.columns = [f"{col}_{key}" for key in dict_test.columns]
            test = pd.concat([test, dict_test], axis=1)
            test.drop(columns=[col], inplace=True)
            
    for col in train.columns:
        if isinstance(train[col].iloc[0], list):
            unique_attributes = set([item for sublist in train[col] for item in sublist])
            for attribute in unique_attributes:
                train[attribute] = train[col].apply(lambda x: 1 if attribute in x else 0)
            train.drop(columns=[col], inplace=True)
    
    for col in test.columns:
        if isinstance(test[col].iloc[0], list):
            unique_attributes = set([item for sublist in test[col] for item in sublist])
            for attribute in unique_attributes:
                test[attribute] = test[col].apply(lambda x: 1 if attribute in x else 0)
            test.drop(columns=[col], inplace=True)
            
    for col in train:
        train[col] = train[col].apply(lambda x: np.nan if (x == {} or x == []) else x)
    for col in test:
        test[col] = test[col].apply(lambda x: np.nan if (x == {} or x == []) else x)
        
    for col in train:
        if train[col].dtype == object:
            train[col] = train[col].apply(lambda x: extract_numeric(x) if not isinstance(x, (list, dict)) else np.nan)
    for col in test:
        if test[col].dtype == object:
            test[col] = test[col].apply(lambda x: extract_numeric(x) if not isinstance(x, (list, dict)) else np.nan)

    single_value_cols = [
        col for col in train.columns
        if train[col].apply(lambda x: tuple(x) if isinstance(x, list) else x).nunique() <= 1
    ]

    single_value_cols = [col for col in single_value_cols if col != "Class"]

    train.drop(columns=single_value_cols, inplace=True)
    test.drop(columns=[col for col in single_value_cols if col in test.columns], inplace=True)

    common_columns = list(set(train.columns) & set(test.columns))
    train = train[common_columns + ["Class"]]
    test = test[common_columns]

    for col in train.columns:
        mode_value = train[col].mode()
        if not mode_value.empty:
            mode_value = str(mode_value[0])
        else:
            mode_value = np.nan

        train[col].fillna(mode_value, inplace=True)
        if col != "Class":
            test[col].fillna(mode_value, inplace=True)

    train = train.reindex(sorted(train.columns), axis=1)
    test = test.reindex(sorted(test.columns), axis=1)
    
    return train, test

train_df, test_df = preprocess_data(train_df, test_df, ["Distribution", "Habits", "Mating_Habits", "Population"])


def label_encode(X):
    label_encoders = {}
    for col in X.select_dtypes(include=['object']):
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col].astype(str))
        label_encoders[col] = le
    return label_encoders

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
آموزش مدل
</font>
</h2>

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    حال که داده را پاکسازی کرده‌اید، وقت آن است که مدلی آموزش دهید که بتواند متغیر هدف این مسئله را پیش‌بینی کند.
</font>
</p>

In [4]:
X_train = train_df.drop(columns=['Class'])
y_train = train_df['Class']

label_encoders = label_encode(X_train)

model = RandomForestClassifier(n_estimators=150, random_state=42)
model.fit(X_train, y_train)

# model = MLPClassifier(hidden_layer_sizes=(128, 64), activation='relu', max_iter=1000, random_state=42)
# model.fit(X_train, y_train)

# model = KNeighborsClassifier(n_neighbors=5)
# model.fit(X_train, y_train)

# model = AdaBoostClassifier(n_estimators=150, random_state=42)
# model.fit(X_train, y_train)

X_test = test_df
for col in X_test.select_dtypes(include=['object']):
    le = label_encoders.get(col, None)
    if le:
        X_test[col] = X_test[col].apply(lambda x: le.transform([x])[0] if x in le.classes_ else -1)

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معیار ارزیابی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    معیاری که برای ارزیابی عملکرد مدل انتخاب کرده‌ایم، <code>f1_score</code> نام دارد.
    <br>
    این معیار، سنجه ارزیابی کیفیت مدل شماست. به عبارت بهتر در سامانه داوری هم از همین معیار برای نمره‌دهی استفاده شده است.
    <br>
    پیشنهاد می‌شود با توجه به این معیار، عملکرد مدل خود را بر روی مجموعه‌ی آموزش یا اعتبارسنجی ارزیابی کنید.
</font>
</p>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font color="red"><b color='red'>توجه:</b></font>
<font face="vazir" size=3>
    برای دریافت نمره از این سوال لازم است تا دقت مدل شما از آستانه‌ی ۰.۴ بیشتر باشد.
    در صورتی که دقت مدل شما از ۰.۴ کمتر باشد نمره شما 
    <b>صفر</b>
    خواهد شد و در غیر این صورت با فرمول زیر محاسبه می‌شود:
</font>
</p>


In [5]:
from sklearn.metrics import f1_score
# Evaluate your model
# TODO

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
 پیش‌بینی بر روی داده‌ی تست و خروجی
</font>
</h2>

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    پیش‌بینی مدل خود بر روی داده‌های آزمون را در یک دیتافریم (<code>dataframe</code>) به فرمت زیر ذخیره کنید.
</font>
</p>


<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    توجه داشته باشید که نام دیتافریم باید <code>submission</code> باشد؛ در غیر این‌صورت، سامانه‌ی داوری قادر به ارزیابی خروجی شما نخواهد بود.
    این دیتافریم صرفا شامل یک ستون به نام <code>Class</code> است و ۱۷۰۰ سطر دارد.
    <br>
    به ازای هر سطر موجود در مجموعه‌داده‌ی آزمون، باید یک کلاس پیش‌بینی‌شده داشته باشید و مقدار <code>Class</code> پیش‌بینی مدل شما برای آن سطر است.
    به‌عنوان مثال جدول زیر، ۵ سطر ابتدایی دیتافریم <code>submission</code> را نشان می‌دهد. البته این مقادیر به‌صورت فرضی هستند و در جواب شما، ممکن است متفاوت باشند.
</font>
</p>

<center>
<div align=center 
style="direction: ltr;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    
||<code>Class</code>|
|:----:|:-----:|
|0|Gastropoda|
|1|Amphibia|
|2|Amphibia|
|3|Aves|
|4|Amphibia|

</font>
</div>
</center>

In [6]:
submission = model.predict(X_test)
submission = pd.DataFrame(submission, columns=["Class"])

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
<b>سلول جواب‌ساز</b>
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    برای ساخته‌شدن فایل <code>result.zip</code> سلول زیر را اجرا کنید. توجه داشته باشید که پیش از اجرای سلول زیر تغییرات اعمال شده در نت‌بوک را ذخیره کرده باشید (<code>ctrl+s</code>) در غیر این صورت، در پایان مسابقه نمره شما به صفر تغییر خواهد کرد.
    <br>
    همچنین اگر از گوگل کولب برای اجرای این فایل نوت‌بوک استفاده می‌کنید، قبل از ارسال فایل <code>result.zip</code>، آخرین نسخه‌ی نوت‌بوک خود را دانلود کرده و داخل فایل ارسالی قرار دهید.
</font>

In [7]:
import zipfile
import os

if not os.path.exists(os.path.join(os.getcwd(), 'SharifML_Contest_ML.ipynb')):
    %notebook -e SharifML_Contest_ML.ipynb

def compress(file_names):
    print("File Paths:")
    print(file_names)
    compression = zipfile.ZIP_DEFLATED
    with zipfile.ZipFile("result.zip", mode="w") as zf:
        for file_name in file_names:
            zf.write('./' + file_name, file_name, compress_type=compression)

submission.to_csv('submission.csv', index=False)

file_names = ['SharifML_Contest_ML.ipynb', 'submission.csv']
compress(file_names)

File Paths:
['SharifML_Contest_ML.ipynb', 'submission.csv']
